In [ ]:
from cil.framework import AcquisitionGeometry, AcquisitionData

from cil.optimisation.algorithms import FISTA
from cil.optimisation.functions import ZeroFunction, LeastSquares

from cil.io import TIFFStackReader

from cil.processors import TransmissionAbsorptionConverter

from cil.plugins.astra import ProjectionOperator, FBP

from cil.plugins.ccpi_regularisation.functions import FGP_TV

from cil.utilities.display import show2D, show_geometry
from cil.utilities.jupyter import islicer, link_islicer

import numpy as np
import matplotlib.pyplot as plt
import os


# Lamniography reconstruction with TV regularisation using FISTA

This exercise uses CIL to reconstruct a Laminography dataset - that is a dataset with a tilted rotation axis. 

Laminography scanning is commonly used for large thin samples like circuit boards. In conventional CT these samples lead to widely varying path-lengths as the sample rotates. By tilting the sample and rotating it around a vector perpendicular to the sample plane the path-lengths stay relatively constant. However, this geometry leads to some artefacts in the reconstruction from the missing data. These artefacts are particularly prevelant when you look at a slice out of the sample plane.

By using regularisation, we can supress these artefacts. This notebook compares FDK, a least-squares with FISTA and Total-Variation with FISTA on a Lego phantom.

**Learning objectives:**

1. Construct an advanced AcquistionGeometry by-hand to describe the titled and offset data

2. Use this geometry to read in a Tif stack and create an AcquisitionData object

3. Create a custom ImageGeometry around the thin-flat sample

4. Reconstruct the data with LS and TV



## Create the acquisition geometry

We know the system parameters from the paper and author clarification. We use this to set up a 3D cone beam geometry with the rotation axis tilited 30&deg towards the source.


In [ ]:

#parameters are from the original paper/author clarification
src_to_det = 967.3209839
src_to_object = 295
tilt = 30. * np.pi / 180.
centre_of_rotation = 0.254 * 6.

mag = src_to_det / src_to_object 
object_offset_x = centre_of_rotation / mag

source_pos_y = -src_to_object
detector_pos_y = src_to_det-src_to_object
angles_list = -np.linspace(0, 360, 2513, endpoint=False)
num_pixels_x = 1596
num_pixels_y = 1148
pixel_size_xy = 0.254


ag = AcquisitionGeometry.create_Cone3D( source_position=[0.0, source_pos_y,0.0], \
                                        detector_position=[0.0, detector_pos_y,0.0],\
                                        rotation_axis_position=[object_offset_x,0,0],\
                                        rotation_axis_direction=[0,-np.sin(tilt), np.cos(tilt)] ) \
                        .set_angles(angles=angles_list, angle_unit='degree')\
                        .set_panel( num_pixels=[num_pixels_x, num_pixels_y], \
                                    pixel_size=pixel_size_xy,\
                                    origin='top-left')\
                        .set_labels(['angle','vertical','horizontal'])
print(ag)
show_geometry(ag)

## Read in the data
We will start by reading in data from a stack of tiffs. As we read in the data we will symetrically crop and down-sample the data for speed of running this example.

As we are reading in a stack of tiffs we know that 'axis_0' is the angle, 'axis_1' is vertical and 'axis_2' is the horizontal. We want to down-sample and crop only in the horizontal and vertical directions. We create a dictionary of the axis name, and a tuple with the the starting pixel index, the end pixel index and the step size to bin.

We also need to change out geometry to account for the new panel size.



In [ ]:
binning = 4
crop = 100

roi = {'axis_1': (crop, -crop, binning), 
       'axis_2': (crop, -crop, binning)}

num_pixels_x = (1596 -2*crop) / binning
num_pixels_y = (1148 -2*crop) / binning
pixel_size_xy = 0.254 * binning

ag.set_panel(num_pixels=[num_pixels_x,num_pixels_y],pixel_size=pixel_size_xy)


       

From `cil.io` we import and create a `TIFFStackReader` instance to read in the data, this is created with the directory path and the RoI dictionary defined above.

As we have already defined our aquisition geometry we can use the function `read_as_AcquisitionData()` to pass this to the reader which will use this to configire and return an `AcquisitionData` object containing the data and the geometry describing it. 

In [ ]:
path_common = '/home/tpc56154/Data/LegoLaminography'
path = 'Lego_Lamino30deg_XTH/'

reader = TIFFStackReader(file_name=os.path.join(path_common, path),roi=roi)
acq_data_raw = reader.read_as_AcquisitionData(ag)


islicer(acq_data_raw, direction='angle',origin='upper-left')


We now read in the dark and flat field images and use these to normalise the data. We use `read()` here to simply read in the tiffs and return a numpy array which we delete after applying to the data.

In [ ]:
tiffs = [   os.path.join(path_common,'Lego_Lamino30deg_ShadingCorrection_XTH/Dark_80kV85uA.tif'),
            os.path.join(path_common,'Lego_Lamino30deg_ShadingCorrection_XTH/Flat_80kV85uA.tif') ]

reader = TIFFStackReader(file_name=tiffs, roi=roi)
dark_flat_data = reader.read()

acq_data_SC = (acq_data_raw-dark_flat_data[0]) / (dark_flat_data[1]-dark_flat_data[0])
islicer(acq_data_SC, direction='angle',origin='upper-left')

del acq_data_raw
del dark_flat_data

Finally we convert the intensity data to attenuation data using the Beer-Lambert law

In [ ]:

converter = TransmissionAbsorptionConverter()
acq_data_atten = converter(acq_data_SC)
islicer(acq_data_atten, direction='angle',origin='upper-left')

del acq_data_SC


## Reconstructing the data using FDK

As we are using the astra backend we need to reorder the data for use by astra. If we wre using TIGRE as the backend we could use `data.reorder('tigre')`

In [ ]:
acq_data = acq_data_atten
acq_data.reorder('astra')
ag = acq_data.geometry
print(ag.dimension_labels)

We use FDK to reconstruct the data. We can use this output to shrink the reconstruction volume defined by `ImageGeomerty` to a tight region around the object. 

In [ ]:
ig = ag.get_ImageGeometry()
# ig.voxel_num_z=100
# ig.voxel_num_y=250

fbp = FBP(ig, ag)
FDK_reco = fbp(acq_data)


In [ ]:
show2D(FDK_reco,slice_list=[('vertical',50),('horizontal_y',125)], title="FBP reconstruction", fix_range=(-0.02,0.07))

In [ ]:
islicer(FDK_reco, direction=vertical')

In [ ]:
show_geometry(ag,ig)

In [ ]:
##%% Now we hate set up the geometry! Time to compare reconstructions. We'll run Fista iwith LS, and Fista with TV and non-negativity. Both will have a warm start from the FDK reconstuction so will need fewer iterations

In [ ]:
Projector = ProjectionOperator(ig, ag)
LS = LeastSquares(A=Projector, b=acq_data)

if binning == 4: #pre-calculated for your convinience
    LS.L = 80744.14062500001
print("Lipschitz constant =", LS.L)


In [ ]:
fista_LS = FISTA(initial=FDK_reco, f=LS, g=ZeroFunction(), max_iteration=1000, update_objective_interval=10)


In [ ]:
fista_LS.run(100)
LS_reco = fista_LS.get_output()

show2D(LS_reco,slice_list=[('vertical',40),('horizontal_y', 139)], title="LS reconstruction", fix_range=(-0.02,0.07))

plt.figure()
plt.semilogy(fista_LS.objective)
plt.title('FISTA LS criterion')
plt.show()


In [ ]:
alpha = 0.5
TV = alpha*FGP_TV(device='gpu')


In [ ]:
fista_TV = FISTA(initial=FDK_reco, f=LS, g=TV, max_iteration=1000, update_objective_interval=10)
fista_TV.update_objective_interval = 10


In [ ]:
fista_TV.run(100)
TV_reco = fista_TV.get_output()
show2D(TV_reco,slice_list=[('vertical',40),('horizontal_y',139)], title="TV reconstruction", fix_range=(-0.02,0.07))

plt.figure()
plt.semilogy(fista_TV.objective)
plt.title('FISTA TV criterion')
plt.show()



In [ ]:

h1 = islicer(FDK_reco,direction='horizontal_y', minmax=(-0.02,0.07))
h2 = islicer(LS_reco,direction='horizontal_y', minmax=(-0.02,0.07))
h3 = islicer(TV_reco,direction='horizontal_y', minmax=(-0.02,0.07))

link_islicer(h1,h2,h3)
print("fin")